In [1]:
'''Helper'''
import pandas as pd
import numpy as np
import time
import joblib
import pickle
import re
import string
import unicodedata
import matplotlib.pyplot as plt
from numpy import argmax

'''Models'''
from transformers import pipeline
# from onnx_transformers import pipeline # to run this -- downgrade transformers library to 3.2.0

'''Display'''
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [2]:
import scipy
# import spacy
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import torch

#Device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

import transformers
print(transformers.__version__)

4.5.1


In [5]:
!nvidia-smi

Thu Sep  2 16:16:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    25W /  75W |      2MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla P4


In [8]:
print(torch.cuda.memory_summary(device=0, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

### Input data

In [9]:
df = pd.read_pickle("preprocess_mnet_train.pkl")
df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()

(11459, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7
1,blogandpostid/blog/post/4587-60d354bc9690ca000...,2021-06-23,Food and Drink,Travel,Travel Type,Celebrate The Lodge At Woodloch’s 15-Year Anni...,celebrate lodge woodloch's 15-year anniversary...,23
2,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12
3,blogandpostid/blog/post/5259-6015f002c7824f000...,2021-02-09,Travel,Shopping,Gifts and Greetings Cards,Valentine’s Day Gift Guide: The Most Romantic ...,valentine's day gift guide: romantic hotel pac...,13
4,blogandpostid/blog/post/4983-5f8064d69a4142000...,2020-10-14,Shopping,Shopping,Sales and Promotions,The Best Prime Day Bike Deals For Saving Big o...,best prime day bike deals saving big bicycles ...,13


In [10]:
df.clean_summary.str.len().describe().apply(int)

count    11459
mean      3597
std       2125
min        433
25%       2231
50%       3155
75%       4333
max      30344
Name: clean_summary, dtype: int64

In [11]:
test_5 = df.clean_summary[:5].tolist()
test_1 = df.clean_summary[0]
candidate_labels = df.tier1.unique().tolist()
print(len(candidate_labels))

28


In [12]:
classifier = pipeline("zero-shot-classification", framework="pt", device=0)

In [13]:
import torch, gc

start_time = time.time()

results = []

for i in range(len(test_5)):
    gc.collect()
    torch.cuda.empty_cache()
    results.append(classifier(test_5[i][:1500], candidate_labels, multi_label=True))
    
print("query time (mins) - ", round((time.time() - start_time)/60, 2))

query time (mins) -  0.19


In [14]:
results

[{'sequence': "wework accept crypto pay landlords crypto. topline wework start accepting payment cryptocurrencies intends hold assets balance sheet, office sharing giant announced tuesday, similar elon musk's tesla march said retain bitcoin purchase cars signal confidence cryptocurrencies inch mainstream acceptance. key facts wework said accept bitcoin, ethereum, usd coin, paxos cryptocurrencies payment offerings, including memberships. company said retain payments balance sheet converting currency look pay landlords partners crypto amenable. coinbase, usa' largest crypto exchange went public l ast week, client pay wework membership cryptocurrency, statement said. wework chief executive sandeep mathrani, said company needs adapt needs growing membership fintech sector service new economy. key background cryptocurrencies increasingly popular form payment. elon musk's tesla, holds sizable investment currency, attracted headlines announced accept bitcoin payment cars. venmo announced supp

In [15]:
print(torch.cuda.memory_summary(device=0, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    1553 MB |    3896 MB |  196933 MB |  195379 MB |
|       from large pool |    1552 MB |    3895 MB |  196906 MB |  195353 MB |
|       from small pool |       1 MB |       2 MB |      27 MB |      25 MB |
|---------------------------------------------------------------------------|
| Active memory         |    1553 MB |    3896 MB |  196933 MB |  195379 MB |
|       from large pool |    1552 MB |    3895 MB |  196906 MB |

**Requirements.txt**

git+https://github.com/huggingface/transformers.git
* below was to speed up zero-shot transform pipeline. Works only with transformers==3.2.0. 
* Had to downgrade to my "transformers" library to test. Dint work out i.e. saw no speed increase in getting zero-shot sentiments
* So upgraded transformers again to 4.5.1 <br>

git+https://github.com/patil-suraj/onnx_transformers  

**Zero-shot classification**<br>
[Ref1](https://www.kaggle.com/foolofatook/zero-shot-classification-with-huggingface-pipeline) [Ref2](https://medium.com/@b.terryjack/zero-shot-classification-584bb39da2e2) <br>
[This is very slow](https://twitter.com/joeddav/status/1363543296166002688) [Slow 2](https://discuss.huggingface.co/t/way-to-make-inference-zero-shot-pipeline-faster/1384/11)
<br> 
**To get sentiment of tweets**
See engagement by sentiment and subjectivity for Forbes and others

In [ ]:
classifier = pipeline("zero-shot-classification")
# TUTORIAL

seq = "The GDP of many countries have been affected by this pandemic."
candidate_labels = ["politics", "public health", "economics"]

classifier(seq, candidate_labels)

sequences = [
    "Tenet is simply an incredible film with deep complex concepts to unravel well after the credits roll.",
    "The Social Dilemma is densely packed yet lively and entertaining documentary"
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

In [18]:
# with ONNX pipeline to speed up the process
test1_zero_shot = pipeline("zero-shot-classification", onnx=True)

def test1_sentiment(txt):
    results =  test1_zero_shot(txt, ["positive", "neutral", "negative"]) #, multi_label=False
    
    scores = results["scores"]
    classes = results["labels"]
    best_index = argmax(scores)
    predicted_class = classes[best_index]
    return predicted_class

In [19]:
start_time = time()

forbes_df[:100].clean_text.apply(test1_sentiment)

print("--- %s seconds ---" % ((time() - start_time)/60))

--- 0.7903248349825541 seconds ---


In [21]:
start_time = time()

forbes_df[:200].clean_text.apply(test1_sentiment)

print("--- %s seconds ---" % ((time() - start_time)/60))

--- 1.6331705451011658 seconds ---


In [9]:
start_time = time()

forbes_df[:100].clean_text.apply(test1_sentiment)

print("--- %s seconds ---" % ((time() - start_time)/60))

--- 0.7834308664004008 seconds ---


In [ ]:
# , device=0 -- needs NVIDIA GPU and driver installed
zero_shot_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", framework="pt") 

def get_sentiment(txt):
    results = zero_shot_classifier(
                            txt, 
                            ["positive", "neutral", "negative"] #multi_label=False
                            )
    
    scores = results["scores"]
    classes = results["labels"]
    best_index = argmax(scores)
    predicted_class = classes[best_index]
    return predicted_class

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [7]:
zero_shot_classifier

In [8]:
start_time = time()

forbes_df[:100].clean_text.apply(get_sentiment)

print("--- %s seconds ---" % ((time() - start_time)/60))

--- 0.29568695624669394 seconds ---


In [15]:
start_time = time()

forbes_df[:200].clean_text.apply(get_sentiment)

print("--- %s seconds ---" % ((time() - start_time)/60))

ERROR! Session/line number was not unique in database. History logging moved to new session 107
--- 0.9960396806399028 seconds ---


**Abstract summarization**

In [2]:
from transformers import pipeline
import os

In [3]:
summarizer = pipeline("summarization")

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]